In [ ]:
#  Google Drive（讀取知識庫與 CSV 資料）
from google.colab import drive
drive.mount('/content/drive')

# 安裝所需套件
!pip install flask line-bot-sdk pyngrok openai flask-cors pandas


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.5 MB/s eta 0:00:00


In [ ]:
# 設定環境變數（建議手動輸入）
import os
os.environ['LINE_CHANNEL_SECRET'] = '你的 LINE Channel Secret'
os.environ['LINE_CHANNEL_TOKEN']  = '你的 LINE Channel Access Token'
os.environ['OPENAI_API_KEY']     = '你的 OpenAI API Key'
os.environ['NGROK_AUTHTOKEN']    = '你的 ngrok Authtoken'


In [ ]:
# 載入套件與環境變數
import os
import json
import pandas as pd
from flask import Flask, request, abort
from flask_cors import CORS
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import MessageEvent, TextMessage, TextSendMessage
from pyngrok import ngrok
import openai

# 讀取環境變數
LINE_SECRET = os.getenv('LINE_CHANNEL_SECRET')
LINE_TOKEN  = os.getenv('LINE_CHANNEL_TOKEN')
OPENAI_KEY  = os.getenv('OPENAI_API_KEY')
NGROK_TOKEN = os.getenv('NGROK_AUTHTOKEN')
openai.api_key = OPENAI_KEY

#載入知識庫與 CSV 資料
kb_path  = '/content/drive/MyDrive/Flask/knowledge_base.txt'
if os.path.exists(kb_path):
    with open(kb_path, 'r', encoding='utf-8') as f:
        knowledge_base = f.read()
else:
    knowledge_base = '知識庫檔案未找到，使用預設內容。'

# 定義 function-calling 的 JSON schema
functions = [
    {
        "name": "generate_structured",
        "description": "回傳符合指定 schema 的結構化回應，僅輸出 JSON",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "回應的標題"
                },
                "steps": {
                    "type": "array",
                    "items": {"type": "string"},
                    "description": "步驟或重點清單"
                },
                "notes": {
                    "type": "string",
                    "description": "額外備註（可選）"
                }
            },
            "required": ["title", "steps"]
        }
    }
]

# 定義取得結構化回應的函式
def get_structured_response(user_input, previous=None):
    messages = [
        {"role": "system", "content": "Knowledge Base:\n" + knowledge_base},
        {"role": "system", "content": "請根據指定的 JSON schema 回傳結構化回應，且僅輸出 JSON，不要其他文字。"}
    ]
    if previous:
        messages.append({"role": "assistant", "content": previous})
    messages.append({"role": "user", "content": user_input})

    resp = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        functions=functions,
        function_call={"name": "generate_structured"},
        max_tokens=250,
        temperature=0.7,
        frequency_penalty=0.8,
        presence_penalty=0.6
    )

    args_str = resp.choices[0].message.function_call.arguments
    return json.loads(args_str)

# 初始化 Flask 與 LINE Bot
app = Flask(__name__)
CORS(app)
line_bot_api = LineBotApi(LINE_TOKEN)
handler      = WebhookHandler(LINE_SECRET)
BOT_NAME = "貼心小助理"  

# Webhook 路由
@app.route('/callback', methods=['POST'])
def callback():
    signature = request.headers.get('X-Line-Signature', '')
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

# 處理訊息
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    user_text   = event.message.text
    source_type = event.source.type
    mention_tag = f"@{BOT_NAME}"
    should_reply = False
    clean_text   = ""

    # 私聊回覆
    if source_type == 'user':
        should_reply = True
        clean_text = user_text.strip()

    # 群組/聊天室需 @ 才回覆
    elif source_type in ['group', 'room'] and mention_tag.lower() in user_text.lower():
        should_reply = True
        # 移除第一個 @
        clean_text = user_text.replace(mention_tag, "", 1).strip()
        if not clean_text:
            # 處理不同大小寫或其他
            for word in user_text.split():
                if word.lower().startswith(f"@{BOT_NAME.lower()}"):
                    clean_text = user_text.replace(word, "", 1).strip()
                    break

    # 如果符合條件呼叫 GPT 回覆
    if should_reply and clean_text:
        structured = get_structured_response(clean_text)
        title = structured.get("title", "")
        steps = structured.get("steps", [])
        notes = structured.get("notes", "")

        # 組成純文字回覆
        reply_lines = []
        if title:
            reply_lines.append(f"📌 {title}")
        for idx, step in enumerate(steps, start=1):
            reply_lines.append(f"{idx}. {step}")
        if notes:
            reply_lines.append(f"\n💡 備註：{notes}")

        reply_text = "\n".join(reply_lines)
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=reply_text)
        )

<ipython-input-4-043eb2bf5763>:82: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(LINE_TOKEN)
<ipython-input-4-043eb2bf5763>:83: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler      = WebhookHandler(LINE_SECRET)


In [ ]:
if __name__ == '__main__':
    ngrok.set_auth_token(NGROK_TOKEN)
    public_url = ngrok.connect(5000).public_url
    print(f'請將此 URL 填入 LINE Developers → Webhook URL：{public_url}/callback')
    app.run(host='0.0.0.0', port=5000)

請將此 URL 填入 LINE Developers → Webhook URL：https://209d-34-27-77-131.ngrok-free.app/callback
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
<ipython-input-4-043eb2bf5763>:140: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
INFO:werkzeug:127.0.0.1 - - [16/May/2025 06:52:54] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 06:52:56] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 06:53:01] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/May/2025 06:53:23] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 -